In [1]:
import sys
sys.path.append('../Output Files')
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import VideoFileClip, AudioFileClip
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
def extract_audio(video_file):
    """Function to extract audio from a video file and save it as an mp3"""
    try:
        # load video
        clip = VideoFileClip(video_file)

        # extract audio
        audio = clip.audio

        # save audio as mp3
        audio.write_audiofile('../Output Files/Audio.mp3')

        return 200

    except:
        pass


In [3]:
def generate_video_w(video_file):
    
    """Function to create a video with 1080 px width and n height"""
    
    try:
        # Saving the audio of video as mp3
        extract_audio(video_file)
        
        # load video
        cap = cv2.VideoCapture(video_file)

        # get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('../Output Files/Bardotics W Video.mp4', fourcc, fps, (1080, height))

        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                # create a blank canvas
                canvas = np.zeros((height, 1080, 3), dtype=np.uint8)

                # calculate new width of the frame
                new_width = 1066 - 14

                # resize frame
                frame = cv2.resize(frame, (new_width, height))

                # place the frame onto the canvas
                canvas[:, 14:14+new_width] = frame

                # write the frame
                out.write(canvas)
            else:
                break

        # release everything
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        
        return 200
    
    except Exception as e:
        print(str(e))
        return 401

In [4]:

def generate_header_image(text):
    
    """Function to generate Header image with logo"""
    
    try:
    
        # Load the image
        image_path = r"../Hardcodes/Templates/Bardotics header.png"  # Replace with the path to your image file


        background = Image.open(image_path)

        # Load the font
        font_path = r"../Hardcodes\Fonts\GothamBook.ttf"  # Replace with the path to your Gotham font file
        font_size = 35
        font = ImageFont.truetype(font_path, int(font_size))

        # Create a draw object
        draw = ImageDraw.Draw(background)

        # Set the text and color
        text_color = (255, 255, 255)  # White color

        # Set the starting and ending columns for the text
        start_col = 14
        end_col = 1066

        # Calculate the width and height of the text
        tw, text_height = draw.textsize(text, font=font)

        # Calculate the width of the slice of the background
        background_slice_width = end_col - start_col

        # Split the text into words
        words = text.split()

        # Initialize a list to hold the lines of text
        lines = []
        line = ""

        # Create lines of text that fit within the specified width
        for word in words:
            if draw.textsize(line + word, font=font)[0] <= background_slice_width:
                line += word + " "
            else:
                lines.append(line)
                line = word + " "
        lines.append(line)

        # Set the line spacing
        line_spacing = 12  # Change this value to adjust the line spacing

        # Calculate the total height of the text
        total_text_height = len(lines) * (text_height + line_spacing)

        # Calculate the starting y position to center the text vertically
        start_y = 106

        # Draw each line of text on the image
        for i, line in enumerate(lines):
            y = start_y + i * (text_height + line_spacing)
            draw.text((start_col, y), line, fill=text_color, font=font)

        # Calculate the cutting pixel (Bottom)
        cutting_px = y + text_height + 26

        # Cut the image
        background = background.crop((0, 0, background.width, cutting_px))

        # Save the image
        background.save('..\Output Files\Bardotics H Img.png')
        
        return 200
    except Exception as e:
        print(str(e))
        return 401



In [5]:
def combine_img_vid():
    
    try:

        # specify video file path
        video_file = "../Output Files/Bardotics W Video.mp4"

        # specify image file path
        image_file = "../Output Files/Bardotics H Img.png"

        # load video
        cap = cv2.VideoCapture(video_file)

        # get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # load image
        image = cv2.imread(image_file)

        # resize image to match video width
        image = cv2.resize(image, (width, image.shape[0]))

        # create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(r"../Output Files/Bardotics Combined Video.mp4", fourcc, fps, (width, height + image.shape[0]))

        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                # create a blank canvas
                canvas = np.zeros((height + image.shape[0], width, 3), dtype=np.uint8)

                # place the image onto the canvas
                canvas[:image.shape[0], :] = image

                # place the frame onto the canvas
                canvas[image.shape[0]:, :] = frame

                # write the frame
                out.write(canvas)
            else:
                break

        # release everything
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        
        return 200
    except Exception as e:
        print(str(e))
        return 401


In [6]:
def generate_final_video():

    try:
        # specify video file path
        video_file = r"../Output Files/Bardotics Combined Video.mp4"

        # load video
        cap = cv2.VideoCapture(video_file)

        # get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # create a blank canvas
        canvas = np.zeros((1920, 1080, 3), dtype=np.uint8)

        # create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(r"../Output Files/Bardotics Reel.mp4", fourcc, fps, (1080, 1920))

        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                # calculate the y-coordinate to place the frame at the center of the canvas
                y = (1920 - height) // 2

                # place the frame onto the canvas
                canvas[y:y+height, :] = frame

                # write the frame
                out.write(canvas)

                # clear the canvas for the next frame
                canvas.fill(0)
            else:
                break

        # release everything
        cap.release()
        out.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print(str(e))
        return 401


In [7]:
def add_audio_to_final_video():
    """Function to add an audio track to a video file"""
    
    audio_file = '../Output Files/Audio.mp3'
    video_file = '../Output Files/Bardotics Reel.mp4'
    
    if os.path.exists(audio_file):
        try:

            # load video
            video = VideoFileClip(video_file)

            # load audio
            audio = AudioFileClip(audio_file)

            # add audio to video
            video_with_audio = video.set_audio(audio)

            # save video with new audio
            video_with_audio.write_videofile('../Output Files/Bardotics Reel Final.mp4', codec='libx264')


        except Exception as e:
            print(e)
            pass
    
    return 200

In [8]:
def generate_reel(video_file, text):
    
    try:
    
        w_gen_status = generate_video_w(video_file)

        img_gen_status = generate_header_image(text)

        combine_status = combine_img_vid()

        reel_gen_status = generate_final_video()
        
        add_audio_to_final_video()
        
        return 200
        
    except Exception as e:
        print(str(e))
        return 401

In [9]:
video_file = r"C:\Users\mnsnn\Downloads\CUbe5bHBIM9n8Ld0.mp4"
text = "I had a boss who told his kids that the way to tell when they were through vacuuming was when the vacuum cleaner head had touched every bit of floor."

generate_reel(video_file, text)

MoviePy - Writing audio in ../Output Files/Audio.mp3


MoviePy - Done.
Moviepy - Building video ../Output Files/Bardotics Reel Final.mp4.
MoviePy - Writing audio in Bardotics Reel FinalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../Output Files/Bardotics Reel Final.mp4



Moviepy - Done !
Moviepy - video ready ../Output Files/Bardotics Reel Final.mp4


200